In [2]:
import requests 
import bs4 
import pandas as pd
import re
from requests.exceptions import MissingSchema

In [101]:
def extract_info(sp):
    lst = sp.findAll(['h2','h3', 'a'], {'class':['title','name', 'twitter']})
    name_info = dict()
    for i in lst:
        if i['class'][0] == 'name': 
            name = i.text
            name_info[i.text] = dict()
            name_info[i.text]['href'] = i.find('a')['href']
            name_info[i.text]['title'] = []
            continue
        if i['class'][0] == 'title':
            name_info[name]['title'].append(i.text)
            continue
        if i['class'][0] == 'twitter':
            name_info[name]['twitter'] = i.text.split('\n')[1].strip('@')
        else: name_info[name]['twitter'] = ''
    return name_info

In [102]:
total_data = dict()
for i in range(100):
    url = f'https://www.brookings.edu/experts/page/{i}?'
    r = requests.get(url)
    if not r: break
    soup = bs4.BeautifulSoup(r.text, "html.parser").body
    add = extract_info(soup)
    total_data.update(add)

In [103]:
len(total_data)

426

In [107]:
name = list(total_data.keys())
twitter = [total_data[i]['twitter'] if 'twitter' in total_data[i] else '' for i in total_data]
title = ['\n'.join(total_data[i]['title']) for i in total_data]
user_data = pd.DataFrame({'name':name, 'twitter':twitter, 'title':title})

In [109]:
user_data.to_csv('brookings_twitter.csv', index=False)